### Add word group counts to dataframes in this script

Analyzing saved lyrics dataframes by counting occurences of words of topcis within songs within genres

Then sum the occurences for each artist and for each genre

In [2]:
import pandas as pd
import re
import os
import plotly
import plotly.express as px

### Specify thematic word groups

In [3]:
to_count_groups = [
    {
        'group': 'love',
        'group_words': ['love', 'lover', 'honey', 'baby', 'heart',
                      'sweetheart', 'loverboy', 'babygirl']
    },
    {
        'group': 'money',
        'group_words': ['money', 'cash', 'gold', 'bill', 'check', 'checks',
                     'dolla', 'bag', 'bags']
    },
    {
        'group': 'violence',
        'group_words': ['shoot', 'kill', 'kills', 'victim', 'whack', 'murder',
                      'gun', 'rob', 'robbin', 'steal', 'stole', 'whacked',
                      'killed', 'shot', 'robbed']
    },
    {
        'group': 'drugs',
        'group_words': ['drugs', 'weed', 'kush', 'mary jane',
                      'perc', 'cocaine', 'molly',]
    },
    {
        'group': 'gendered',
        'group_words': ['girl', 'boy', 'man', 'woman', 'him', 'her', 'he', 'she']
    },
    {
        'group': 'sadness',
        'group_words': ['sad', 'sadness', 'hate' ,'hateful', 'sadly', 'melancholy',
                       'sorry', 'bitter', 'heartbreak', 'heartbroken']
    },
    {
        'group': 'joy',
        'group_words': ['joy', 'happy', 'wonder' ,'pride', 'smile', 'joyful',
                       'laugh', 'lucky', 'glad', 'hope', 'hopeful', 'bliss',
                       'euphoria']
    },
    {
        'group': 'yes',
        'group_words': ['yes', 'agree', 'agreed', 'sure', 'confirm', 'accept']
    },
    {
        'group': 'no',
        'group_words': ['no', 'disagree', 'resist', 'reject', 'deny']
    }
]

### Function to count manual group occurrences

In [41]:
def count_occurrences(df, to_count_groups):
    df['unique_words'] = 0
    df['total_words'] = 0
    for i in df.index:
        lyrics = df['Song Lyrics'].str
        lyrics_set = set(df['Song Lyrics'][i].split())
        lyrics_len = len(lyrics_set)
        df['unique_words'][i] = lyrics_len

        lyrics_len_total = len(df['Song Lyrics'][i].split())
        df['total_words'][i] = lyrics_len_total
        
        for group in to_count_groups:
            group_count_total = 0
            for group_word in group['group_words']:
                group_word_count = lyrics.count(group_word)
                group_count_total += group_word_count
                
            df[f"manual_{group['group']}_count"] = group_count_total
    return df

### Create counts for all genres
plus save intermediate within-genre summaries

In [11]:
# define columns to be saved to the new dataframes
count_group_list = []
for group in to_count_groups:
    count_group_list.append(f"manual_{group['group']}_count")
count_group_list

['manual_love_count',
 'manual_money_count',
 'manual_violence_count',
 'manual_drugs_count',
 'manual_gendered_count',
 'manual_sadness_count',
 'manual_joy_count',
 'manual_yes_count',
 'manual_no_count']

In [37]:
# dataframes for genre comparing

to_save_cols = [*count_group_list, 'unique_words', 'total_words',
                'featured_count','producer_count', 'writer_count']

genre_sum_df = pd.DataFrame(columns=to_save_cols)
genre_mean_df = pd.DataFrame(columns=to_save_cols)
print('cols:')
print(list(genre_sum_df.columns))

cols:
['manual_love_count', 'manual_money_count', 'manual_violence_count', 'manual_drugs_count', 'manual_gendered_count', 'manual_sadness_count', 'manual_joy_count', 'manual_yes_count', 'manual_no_count', 'unique_words', 'total_words', 'featured_count', 'producer_count', 'writer_count']


In [14]:
genres = ['pop', 'rap', 'rock', 'soul']

In [42]:
for genre in genres:
    df_path = f'../dataframes/genres/cleaned_{genre}_10.csv'
    df = pd.read_csv(df_path, index_col=0)

    # count occurences and add to dataframes
    df = count_occurrences(df, to_count_groups)

    # save dataframes with counts for each song
    df.to_csv(f'../dataframes/with_counts/{genre}_count.csv')
    

    # group by artists and select only columns with value to us
    df_summed = df.groupby(['Artist']).sum()
    df_summed_cutout = df_summed[to_save_cols]

    df_mean = df.groupby(['Artist'])[to_save_cols].mean()
    df_mean_cutout = df_mean[to_save_cols]
    
    # save within-genre df summed for each artist
    df_len = len(df_summed)
    df_summed_cutout.to_csv(f'../dataframes/group_stats/sum/{genre}_{df_len}.csv')
    df_mean_cutout.to_csv(f'../dataframes/group_stats/mean/{genre}_{df_len}.csv')
    
    genre_sum = pd.DataFrame(df_summed_cutout.sum())
    genre_sum = genre_sum.T
    genre_sum.index = [genre]

    genre_mean = pd.DataFrame(df_mean_cutout.mean())
    genre_mean = genre_mean.T
    genre_mean.index = [genre]

    # add to all genres df
    genre_sum_df = pd.concat([genre_sum_df, genre_sum], axis=0)
    genre_mean_df = pd.concat([genre_mean_df, genre_sum], axis=0)

genre_sum_df.to_csv(f'../dataframes/group_stats/genre_sum_df.csv')
genre_mean_df.to_csv(f'../dataframes/group_stats/genre_mean_df.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_13864\3068466218.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['unique_words'][i] = lyrics_len
C:\Users\User\AppData\Local\Temp\ipykernel_13864\3068466218.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_words'][i] = lyrics_len_total
C:\Users\User\AppData\Local\Temp\ipykernel_13864\3068466218.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['unique_words'][i] = lyr

### Add word percentage count columns

In [43]:
# define columns to be saved to the new dataframes
percent_group_list = []
for group in to_count_groups:
    percent_group_list.append(f"manual_{group['group']}_word_percent")
percent_group_list

['manual_love_word_percent',
 'manual_money_word_percent',
 'manual_violence_word_percent',
 'manual_drugs_word_percent',
 'manual_gendered_word_percent',
 'manual_sadness_word_percent',
 'manual_joy_word_percent',
 'manual_yes_word_percent',
 'manual_no_word_percent']

In [44]:
for genre in genres:
    df_path = f'../dataframes/with_counts/{genre}_count.csv'
    df = pd.read_csv(df_path, index_col=0)

    # prepare columns
    for group in percent_group_list:
        df[group] = 0
    
    for i in df.index:
        for group_index, group in enumerate(percent_group_list):
            df[group][i] = round((df[count_group_list[group_index]][i] / df['total_words'][i])*100,3)

    # save dataframes with counts for each song
    df.to_csv(f'../dataframes/with_counts/{genre}_count.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_13864\3536080180.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[group][i] = round((df[count_group_list[group_index]][i] / df['total_words'][i])*100,3)
C:\Users\User\AppData\Local\Temp\ipykernel_13864\3536080180.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.56' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df[group][i] = round((df[count_group_list[group_index]][i] / df['total_words'][i])*100,3)
C:\Users\User\AppData\Local\Temp\ipykernel_13864\3536080180.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

### Add word percentages to grouped counts
that is, totals for each artist

In [46]:
for genre in genres:
    df_path = f'../dataframes/group_stats/sum/{genre}_10.csv'
    df = pd.read_csv(df_path, index_col=0)

    # prepare columns
    for group in percent_group_list:
        df[group] = 0
    
    for i in df.index:
        for group_index, group in enumerate(percent_group_list):
            df[group][i] = round((df[count_group_list[group_index]][i] / df['total_words'][i])*100,3)

    # save dataframes with counts for each song
    df.to_csv(f'../dataframes/group_stats/sum/{genre}_10.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_13864\3187032168.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2.475' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df[group][i] = round((df[count_group_list[group_index]][i] / df['total_words'][i])*100,3)
C:\Users\User\AppData\Local\Temp\ipykernel_13864\3187032168.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[group][i] = round((df[count_group_list[group_index]][i] / df['total_words'][i])*100,3)
C:\Users\User\AppData\Local\Temp\ipykernel_13864\3187032168.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.313' has dtype incompatible with int64, please expli

### Combine within-genre dataframes with counts into one df

In [32]:
combined_df = pd.DataFrame()
for genre in genres:
    genre_df_path = f'../dataframes/with_counts/{genre}_count.csv'
    genre_df = pd.read_csv(genre_df_path, index_col = 0)
    combined_df = pd.concat([combined_df, genre_df], ignore_index=True)

In [33]:
combined_df.shape

(800, 35)

In [34]:
combined_df.sample(5)

,Artist,Song Title,Full Title,Release Date,Year,Month,Day,Pageviews,url,featured_count,...,manual_no_count,manual_love_word_percent,manual_money_word_percent,manual_violence_word_percent,manual_drugs_word_percent,manual_gendered_word_percent,manual_sadness_word_percent,manual_joy_word_percent,manual_yes_word_percent,manual_no_word_percent
680,Marvin Gaye,What’s Going On,What's Going On by Marvin Gaye,1971-01-17,1971.0,1.0,17.0,638644,https://genius.com/Marvin-gaye-whats-going-on-...,0,...,4,1.770,0.000,0.000,0.0,17.699,0.295,0.000,0.000,1.180
336,Kendrick Lamar,​i (Album Version),​i (Album Version) by Kendrick Lamar,2015-03-15,2015.0,3.0,15.0,2761978,https://genius.com/Kendrick-lamar-i-album-vers...,0,...,25,5.168,1.292,0.775,0.0,31.266,0.000,0.258,0.000,6.460
375,Nas,Daughters,Daughters by Nas,2012-04-26,2012.0,4.0,26.0,353462,https://genius.com/Nas-daughters-lyrics,0,...,15,0.775,0.258,0.517,0.0,32.300,0.000,0.000,0.258,3.876
59,Britney Spears,My Prerogative,My Prerogative by Britney Spears,2004-09-21,2004.0,9.0,21.0,59281,https://genius.com/Britney-spears-my-prerogati...,0,...,5,0.000,0.000,0.000,0.0,3.939,0.000,0.000,0.000,1.094
492,Led Zeppelin,All My Love,All My Love by Led Zeppelin,1979-08-15,1979.0,8.0,15.0,168316,https://genius.com/Led-zeppelin-all-my-love-ly...,0,...,7,10.506,0.000,0.000,0.0,11.673,0.000,0.000,0.778,2.724


In [35]:
combined_df.to_csv(f'../dataframes/with_counts/combined_count.csv')